# Polyvariadic Functions
Let's make a function that builds a list, taking an arbitrary number of arguments.

In [38]:
:ext FunctionalDependencies
:ext MultiParamTypeClasses
:ext FlexibleInstances
:ext RankNTypes
:ext FlexibleContexts

import Data.Functor

-- there can only be one type a corresponding to a type b
class BuildList a r | r -> a where
  build' :: [a] -> a -> r
  
instance BuildList a [a] where
  build' xs x = reverse $ x : xs
  
instance BuildList a r => BuildList a (a -> r) where
  build' xs x y = build' (x:xs) y
  
polyList :: forall a r. BuildList a r => a -> r
polyList = build' []

polyList 1 2 3 :: [Int]
polyList False True True :: [Bool]


[1,2,3]

[False,True,True]

In [36]:
-- :ext TypeFamilies
:ext UndecidableInstances
:ext InstanceSigs

class Variadic a r t | t -> r where
  liftVariadic :: ([a] -> r) -> t

instance Variadic a r r where
  liftVariadic f = f []

instance (a ~ a', Variadic a r t) => Variadic a r (a' -> t) where
  liftVariadic :: ([a] -> r) -> a -> t
  liftVariadic f h = liftVariadic (f . (h:))

polyAdd :: Variadic Int Int t => t
polyAdd = liftVariadic (sum :: [Int] -> Int)

polyWords :: Variadic String String t => t
polyWords = liftVariadic unwords

polyList :: Variadic a [a] t => t
polyList = liftVariadic id

In [28]:
-- build' [1] [2] [3] :: [[Int]]

polyAdd 2 3 4 5 :: Int

sum . (4:) . (3:) $ []

polyWords "test" "task" :: String

14

7

"test task"

String templating example

In [108]:
:ext MultiParamTypeClasses
:ext FunctionalDependencies
:ext FlexibleInstances
:ext FlexibleContexts
:ext UndecidableInstances
:ext InstanceSigs

import qualified Data.Map as M
import Data.Char (isDigit)

class Interpolate a r t | t -> r where
  interpolate' :: ([a] -> r) -> t
  
instance Interpolate a r r where
  interpolate' f = f []
  
instance (a ~ a', Interpolate a r t) => Interpolate a r (a' -> t) where
  interpolate' f a = interpolate' (f . (a:))

interpolate :: (Interpolate String String t) => String -> t
interpolate template = interpolate' go where --interpolate' id where
  go xs = replace template where
    dict = M.fromList $ zip (map show [1..]) xs 
    
    replace [] = []
    replace ('$' : xs)
      | Just w <- M.lookup key dict = w ++ replace ws'
      | otherwise = replace ws'
      where
        (key, ws') = span isDigit xs
    replace (x:xs) = x : replace xs
    

interpolate "template $1 $2 $3 $1" "test" "we" "three" :: String
interpolate "Hello $1, we see that you like $2s. Shop our selection of $2s today!" "Bob" "computer" :: String

"template test we three test"

"Hello Bob, we see that you like computers. Shop our selection of computers today!"

In [17]:
:ext MultiParamTypeClasses
:ext FunctionalDependencies
:ext FlexibleInstances
:ext FlexibleContexts
:ext UndecidableInstances
:ext InstanceSigs

import qualified Data.Map as M
import Data.Char (isDigit)

class Interpolate t where
  interpolate' :: ([String] -> String) -> t
  
instance Interpolate String where
  interpolate' f = f []
  
instance (Interpolate t) => Interpolate (Integer -> t) where
  interpolate' f a = interpolate' (f . (show a :))
  
instance (Interpolate t) => Interpolate (Double -> t) where
  interpolate' f a = interpolate' (f . (show a :))
  
instance (Interpolate t) => Interpolate (String -> t) where
  interpolate' f a = interpolate' (f . (a:))



interpolate :: (Interpolate t) => String -> t
interpolate template = interpolate' go where --interpolate' id where
  go xs = replace template where
    dict = M.fromList $ zip (map show [1..]) xs 
    
    replace [] = []
    replace ('$' : '$' : xs) = '$' : replace xs
    replace ('$' : xs)
      | Just w <- M.lookup key dict = w ++ replace ws'
      | otherwise = replace ws'
      where
        (key, ws') = span isDigit xs
    replace (x:xs) = x : replace xs
    

interpolate "template $1 $2 $3 $1" "test" "we" "three" :: String
interpolate "Hello $1, we see that you like $2s. Shop our selection of $2s today! Save $$4! $3" "Bob" "computer" 3.23 :: String

"template test we three test"

"Hello Bob, we see that you like computers. Shop our selection of computers today! Save $4! 3.23"